# MidTerm_Assignment

### Problem_Statement

You are given the reviews dataset. These are 194439 amazon reviews for cell phones and
accessories taken from https://jmcauley.ucsd.edu/data/amazon/ Use the “reviewText” and
“overall” fields from this file. The goal is to predict the rating given the review by modeling it as
a multi-class classification problem.

1. Take the first 70% dataset for train, next 10% for validation/development, and remaining
20% for test.
2. Traditional machine learning methods
a. Design some good linguistic features. You can start with basic TFIDF features.
Use these classifiers: J48 decision trees, SVMs with linear/RBF kernel, logistic
regression, xgboost, random forests and report accuracy on test set.


### Importing the Required Libraries

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split



#### Reading the data file

In [2]:
# Load the dataset
full_data = pd.read_json("E:/studies/AI_and_Decision_Sciences _2/Mid Term  Assessment/dataset/Cell_Phones_and_Accessories_5.json", lines=True)

# Sample a random subset of the data
sample_size = int(0.005 * len(full_data))
data = full_data.sample(n=sample_size, random_state=42)



In [3]:
data.head()

reviewerID        asin       reviewerName helpful  \
156187  A1P3ACSVXZ1JUM  B00C0ZKI9Q              LO127  [0, 0]   
102252  A2Z3IZDQUIDRR9  B008FQVO5G            yogitim  [8, 8]   
23146   A3EOVXI1VZIHUQ  B003Y74AZ2  Z. Freeman "Zach"  [1, 2]   
86461    AWU526GBZ64MH  B007ZL0TJ4               Bree  [0, 0]   
62407   A3F7F7QKQP2FKT  B005VNK7T0             Beanie  [0, 0]   

                                               reviewText  overall  \
156187  iBOLT xProDock Active Car Dock/Holder/Mount fo...        5   
102252  This pouch is everything that I was looking fo...        5   
23146   First of all - the case I had on my iPhone 4 p...        3   
86461   I ordered this case because I had the same cas...        5   
62407   Bought a few of these as gifts. They were a bi...        5   

                                       summary  unixReviewTime   reviewTime  
156187                          Fits perfectly      1372032000  06 24, 2013  
102252                            Great Pouch!      1361577600  02 23, 2013  
23146   An average iPhone case - nothing fancy      1296604800   02 2, 2011  
86461                               Great Case      1379635200  09 20, 2013  
62407                                     Cute      1371859200  06 22, 2013

In [4]:
# Convert ratings to class labels (0-based)
data['label'] = data['overall'].astype(int) - 1



In [5]:
data.shape

(972, 10)

### Train Test split

In [6]:
# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)



### Tokenization and Dataset

In [7]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a custom dataset class
class ReviewDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        review = str(self.data.iloc[index]['reviewText'])
        label = int(self.data.iloc[index]['label'])
        
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
        }



In [8]:
# Create train and test datasets
train_dataset = ReviewDataset(train_data, tokenizer)
test_dataset = ReviewDataset(test_data, tokenizer)



#### Data Loaders

In [9]:
# Initialize data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)



#### Bert model and Optimizer

In [10]:
# Initialize BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Yugandhar\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Training Loop

In [11]:
# Set device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training parameters
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()  # Set the model to training mode
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    
    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            
            predicted_labels = outputs.logits.argmax(dim=1)
            val_total += labels.size(0)
            val_correct += (predicted_labels == labels).sum().item()
    
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = val_correct / val_total
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Final Evaluation
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_labels = outputs.logits.argmax(dim=1)
        
        test_total += labels.size(0)
        test_correct += (predicted_labels == labels).sum().item()

test_accuracy = test_correct / test_total
print(f"Final Test Accuracy: {test_accuracy:.4f}")


Epoch 1/5, Train Loss: 1.3915, Val Loss: 1.1465, Val Accuracy: 0.5641
Epoch 2/5, Train Loss: 1.2432, Val Loss: 1.1080, Val Accuracy: 0.5641
Epoch 3/5, Train Loss: 1.1923, Val Loss: 1.0602, Val Accuracy: 0.5641
Epoch 4/5, Train Loss: 1.1071, Val Loss: 0.9945, Val Accuracy: 0.5641
Epoch 5/5, Train Loss: 1.0032, Val Loss: 1.0075, Val Accuracy: 0.5692
Final Test Accuracy: 0.5692
